# Analyzing Spain Economy for Elections 2023

## Setting up environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/spain_elections

/content/drive/MyDrive/spain_elections


In [4]:
from IPython.display import clear_output
!pip install requests
!pip install pygwalker -q
clear_output()

In [5]:
from IPython.display import clear_output
import pandas as pd
import numpy as np
import io
from google.colab import files
from IPython.core.display import HTML

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pygwalker as pyg

from IPython.core.display import HTML

# Google Colab
# Enable interactive tables in Google Colab
%load_ext google.colab.data_table

# Disable interactive tables
#%unload_ext google.colab.data_table

from google.colab.data_table import DataTable
DataTable.max_columns = 60


# Setting default Settings
sns.set_style('darkgrid')  # Set default plotting style to 'darkgrid'

import warnings
warnings.filterwarnings("ignore")  # Ignore warning messages


# Set display options for pandas
pd.options.display.max_columns = 100  # Maximum number of columns to display: 100
pd.options.display.max_rows = 100  # Maximum number of rows to display: 100
#pd.options.display.float_format = '{:,.2f}'.format  # Format for floating-point numbers: 4 decimal places, comma as thousands separator
pd.options.display.max_colwidth = 100  # Maximum width for displaying columns: 1000 characters
#pd.options.display.width = 1000  # Maximum display width: 1000 characters (horizontal wrapping)
#pd.options.display.max_seq_items = 1000  # Maximum number of items to display for a sequence: 1000

# import io
# from google.colab import files
# uploaded = files.upload()
#df = pd.read_excel(io.BytesIO(uploaded.get('WEOApr2023all.xlsx')))
#uploaded = files.upload()


## Loading and cleaning dataset

In [6]:
df = pd.read_excel('WEOApr2023all.xlsx')
df = df[~df['WEO Country Code'].isna()]
df = df[~df['ISO'].isna()]
df.columns = df.columns.astype(str)
df.columns = df.columns.str.strip()

# Define the desired column order
desired_columns = [
    'WEO Country Code',
    'ISO',
    'WEO Subject Code',
    'Country',
    'Subject Descriptor',
    'Subject Notes',
    'Units',
    'Scale',
    'Country/Series-specific Notes',
    'Estimates Start After'
] + [str(x) for x in range(1980,2029)]

# Select only the desired columns
df = df[desired_columns]

# xls = pd.read_excel('metadata.xlsx',sheet_name=None)
# print(xls.keys())

xls = pd.read_excel('metadata.xlsx',sheet_name='CONCEPT',skiprows=7,
                   # header=['Code','Description']
                    )
xls.columns = xls.columns.str.strip()
xls = xls[['Code','Description']]
xls = xls.dropna()


df = pd.merge(left=df,right=xls,how="left",left_on="WEO Subject Code",right_on="Code",indicator=True)


df.drop(columns=['_merge','Code'],inplace=True)
df.rename(
    {'Description':'Subject Descriptor2'},axis=1,inplace= True)

year_cols = [str(x) for x in range(1980,2029)]
identifier_cols = [ col for col in df.columns if col not in year_cols]

df = df[identifier_cols+year_cols]

#df.to_csv('data.csv',index=None)

#df = pd.read_csv('data.csv')

df_long = pd.melt(df, id_vars=identifier_cols, var_name='Year', value_name='Value')



# Add 'Estimate' column
df_long['Year'] = df_long['Year'].astype(float)
df_long['Estimate'] = df_long.apply(lambda row: 1 if row['Year'] > row['Estimates Start After'] else 0, axis=1)

df_long = df_long[df_long['Year']>1982]


## Checking for errors

In [7]:
print(f'Duplicates in df set: {df.duplicated().sum()}, ({np.round(100*df.duplicated().sum()/len(df),1)}%)')
print(f'Duplicates in df_long set: {df_long.duplicated().sum()}, ({np.round(100*df_long.duplicated().sum()/len(df_long),1)}%)')


Duplicates in df set: 0, (0.0%)
Duplicates in df_long set: 0, (0.0%)


In [8]:
def multi_table(table_list):
    return HTML(
        f"<table><tr> {''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list])} </tr></table>")

In [9]:
# multi_table([pd.DataFrame(df_long[i].value_counts()) for i in df_long.columns if i != 'Value'])

In [10]:

na_cols=df_long.columns[df_long.isna().any()].tolist()

mv=pd.DataFrame(df_long[na_cols].isna().sum(), columns=['Number_missing'])
mv['Percentage_missing']=np.round(100*mv['Number_missing']/len(df_long),2)
mv

,Number_missing,Percentage_missing
Scale,216384,54.55
Country/Series-specific Notes,45172,11.39
Estimates Start After,45172,11.39
Value,91900,23.17


## Preparing dataset

In [11]:

# # Find the range of years available in the dataset
# year_columns = [col for col in df_eu.columns if col.isdigit()]
# min_year = int(min(year_columns))
# max_year = int(max(year_columns))

# min_year, max_year



# Convert the 'Year' column to numeric
df_long['Year'] = pd.to_numeric(df_long['Year'], errors='coerce')

# Create a function to determine the governing party in Spain based on the year
def get_governing_party(year):
    if 1977 <= year <= 1981:
        return 'UCD'
    elif 1981 < year <= 1982:
        return 'UCD'
    elif 1982 < year <= 1996:
        return 'PSOE'
    elif 1996 < year <= 2004:
        return 'PP'
    elif 2004 < year <= 2011:
        return 'PSOE'
    elif 2011 < year <= 2018:
        return 'PP'
    elif 2018 < year <= 2021:
        return 'PSOE'
    else:
        return None  # for years outside the specified range

# Create the 'Governing Party' column
df_long.loc[df_long['Country'] == 'Spain', 'Governing Party'] = df_long.loc[df_long['Country'] == 'Spain', 'Year'].apply(get_governing_party)

# Drop the rows with estimates

#df_long = df_long[df_long['Year'] < df_long['Estimates Start After']]

# # Display the first few rows of the modified dataframe
# df_long.head()

df_long['Year'] = df_long['Year'].astype(int)

# Define EU country codes based on the provided Excel dataset
eu_country_codes = ['AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK',
                    'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL',
                    'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'POL',
                    'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE']

# Filter data for EU countries

eu_data = df_long[df_long['ISO'].isin(eu_country_codes)]

spain_data = eu_data[eu_data['ISO'] == 'ESP']

## Visualizing Data

### Inflation by Year

In [15]:



data_to_plot = df_long[(df_long['WEO Subject Code'] == 'PCPIPCH') & (df_long['Year']>2000)&(df_long['Estimate']==1)]
# select year range of interest
yr_start = 2001
yr_last = 2021

#df_hist_select = data_to_plot[['Country'] +[str(y) for y in range(yr_start, yr_last+1)]].dropna(how='all')


#df_hist = pd.DataFrame(df_hist_select.set_index('Country').unstack().reset_index())

#df_hist_select
data_to_plot = data_to_plot[['Year','Country','Value','ISO']]

#data_to_plot = data_to_plot[~data_to_plot['Value'].isna()]
data_to_plot['Value'] = pd.to_numeric(data_to_plot['Value'])



fig1 = px.choropleth(data_to_plot,
                     locations="Country",
                     locationmode='country names',
                     color="Value",
                     hover_name="Country",
                     animation_frame="Year",        # column with dates you wish to create animation with
                     color_continuous_scale='PuRd',
                     range_color=(min(data_to_plot['Value']),
                                  max(data_to_plot['Value'])), # to ensure the same colorbar across years
                     height=600,
                     title = "Inflation Rate by Year "
)

fig1.show()

### GDP per capita, constant prices

In [24]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

pio.templates.default = "simple_white"
px.defaults.template = "ggplot2"
px.defaults.color_continuous_scale = px.colors.sequential.Blackbody
px.defaults.width = 1000
px.defaults.height = 1100




spain_gdp_per_capita_nc = eu_data[
    (eu_data['Country'] == 'Spain') &
    (eu_data['WEO Subject Code'] == 'NGDPDPC') &
    (eu_data['Year']>1982) &
    (eu_data['Year']<2023)
]

eu_gdp_per_capita_nc = eu_data[
    #(eu_data['Country'] == 'Spain') &
    (eu_data['WEO Subject Code'] == 'NGDPDPC') &
    (eu_data['Year']>1982) &
     (eu_data['Year']<2023)
]


eu_gdp_per_capita_nc = eu_gdp_per_capita_nc[~eu_gdp_per_capita_nc['Value'].isna()]
spain_gdp_per_capita_nc = spain_gdp_per_capita_nc[~eu_gdp_per_capita_nc['Value'].isna()]

spain_gdp_per_capita_nc['Value']= pd.to_numeric(spain_gdp_per_capita_nc['Value'])
eu_gdp_per_capita_nc['Value'] = pd.to_numeric(eu_gdp_per_capita_nc['Value'])

# Define the governing periods for PP and PSOE
governing_periods = [
    (1982, 1996, 'PSOE', 'red'),
    (1996, 2004, 'PP', 'blue'),
    (2004, 2011, 'PSOE', 'red'),
    (2011, 2018, 'PP', 'blue'),
    (2018, 2021, 'PSOE', 'red'),
]

# Create a subplot for each unique year with increased vertical spacing
fig = make_subplots(rows=2, cols=1, subplot_titles=("GDP per Capita Over Time in Spain", "Average GDP per Capita in Top 20 EU Countries"),
                    vertical_spacing=0.2)  # Increase the vertical spacing

# Add a scatter trace for Spain's GDP per capita over time
fig.add_trace(
    go.Scatter(x=spain_gdp_per_capita_nc['Year'], y=spain_gdp_per_capita_nc['Value'], mode='lines', name='Spain', line=dict(color='black', width=1.5)),
    row=1, col=1
)

# Add shading for governing periods
for start, end, party, color in governing_periods:
    fig.add_vrect(x0=start, x1=end, fillcolor=color, opacity=0.2, row=1, col=1)

# Add legend items (PSOE and PP) to the first map
fig.add_trace(
    go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color='red'), legendgroup='Governing Party', showlegend=True, name='PSOE'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color='blue'), legendgroup='Governing Party', showlegend=True, name='PP'),
    row=1, col=1
)

# Compute the average GDP per capita for each EU country for each year
eu_average_gdp_per_capita = eu_gdp_per_capita_nc.groupby(['Country', 'Year'])['Value'].mean().reset_index()

# Create a bar trace for the average GDP per capita in EU countries for each unique year
for year in eu_average_gdp_per_capita['Year'].unique():
    eu_average_gdp_per_capita_year = eu_average_gdp_per_capita[eu_average_gdp_per_capita['Year'] == year].sort_values('Value', ascending=False).head(20)
    colors = ['orange' if country == 'Spain' else 'lightgray' for country in eu_average_gdp_per_capita_year['Country']]
    fig.add_trace(
        go.Bar(x=eu_average_gdp_per_capita_year['Country'], y=eu_average_gdp_per_capita_year['Value'], name='EU Countries', marker_color=colors, visible=False),
        row=2, col=1
    )

# Define steps for the slider
steps = []
for i, year in enumerate(eu_average_gdp_per_capita['Year'].unique()):
    step = dict(
        method="update",
        args=[{"visible": [i == j for j in range(len(fig.data))]}],
        label=str(year)
    )
    steps.append(step)

# Add a slider to the layout
sliders = [dict(
    yanchor="bottom",  # Anchor the slider to the bottom of the plot
    y=-0.2,  # Adjust the y position of the slider (0.1 corresponds to moving it further down)
    xanchor="left",
    x=0.0,
    active=0,
    currentvalue={"prefix": "Year: "},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    font=dict(
        size=14,  # Increase global font size
    ),
    title=dict(
        text="GDP per Capita in US $",  # Set the main title for the whole chart
        font=dict(
            size=24,  # Increase the title font size
            color="black"  # Set the title color
        )
    ),
    height=1000,  # Set the height of the plot
    width=1000,  # Set the width of the plot
)

# Update x-axis and y-axis for both subplots
for i in range(1, 3):
    fig.update_xaxes(
        tickangle=0 if i == 1 else 30,  # Rotate x-axis labels of the second plot
        tickfont=dict(color='black' if i == 1 else 'black', size=12),  # Decrease x-axis tick font size for the second plot
        row=i, col=1
    )
    fig.update_yaxes(
        tickfont=dict(size=14),  # Increase y-axis tick font size
        showticklabels=False if i == 2 else True,  # Hide y-axis tick labels for the second plot
        row=i, col=1
    )

# Show the plot
fig.show()
